In [1]:
from datetime import datetime
from typing import  Dict
from vnpy.trader.constant import Exchange, Interval
from vnpy.trader.database import database_manager
from vnpy.app.cta_strategy import ArrayManager
from vnpy.chart import ChartWidget, VolumeItem, CandleItem
import pyqtgraph as pg
from vnpy.trader.ui import create_qapp, QtCore, QtGui
from vnpy.trader.object import BarData
from vnpy.chart.manager import BarManager

class BaseIndicatorLine(CandleItem):
    """自定义指标显示"""
    bar_attr_name="down_line"
    rgb_color = (100, 100, 255)
    
    def __init__(self, manager: BarManager):
        """"""
        super().__init__(manager)

        self.blue_pen: QtGui.QPen = pg.mkPen(color=self.rgb_color, width=2)
        self.indicator_data: Dict[int, float] = {}

    def get_indicator_value(self, ix: int) -> float:
        """"""
        if not self.indicator_data:
            bars = self._manager.get_all_bars()
            indicator_array = [getattr(bar, self.bar_attr_name) for bar in bars]

            for n, value in enumerate(indicator_array):
                self.indicator_data[n] = value

        return self.indicator_data.get(ix, 0)


    def _draw_bar_picture(self, ix: int, bar: BarData) -> QtGui.QPicture:
        """"""
        indicator_value = self.get_indicator_value(ix)
        last_indicator_value = self.get_indicator_value(ix - 1)

        # Create objects
        picture = QtGui.QPicture()
        painter = QtGui.QPainter(picture)

        # Set painter color
        painter.setPen(self.blue_pen)

        # Draw Line
        start_point = QtCore.QPointF(ix-1, last_indicator_value)
        end_point = QtCore.QPointF(ix, indicator_value)
        painter.drawLine(start_point, end_point)

        # Finish
        painter.end()
        return picture

    def get_info_text(self, ix: int) -> str:
        """"""
        value = self.indicator_data.get(ix, "-")
        text = f"{self.bar_attr_name} {value}"
        return text



In [2]:
app = create_qapp()

In [ ]:
class ZB(BaseIndicatorLine):
    bar_attr_name="down_line"

    
class ZB2(BaseIndicatorLine):
    bar_attr_name="up_line"

symbol = "000905"
exchange = Exchange.SSE
interval = Interval.DAILY
start = datetime(2020, 1, 1)
end = datetime(2022, 1, 1)

bars = database_manager.load_bar_data(
    symbol=symbol,
    exchange=exchange,
    interval=interval,
    start=start,
    end=end
)

# calculate indicator
am = ArrayManager(50)
for bar in bars:
    am.update_bar(bar)
    bar.up_line, bar.down_line = am.boll(20,2)


widget = ChartWidget()
widget.add_plot("candle", hide_x_axis=True)
widget.add_plot("volume", maximum_height=250)
widget.add_item(CandleItem, "candle", "candle")
widget.add_item(VolumeItem, "volume", "volume")

widget.add_item(ZB, "ZB", "candle")
widget.add_item(ZB2, "ZB2", "candle")
widget.add_cursor()

history = bars
widget.update_history(history)

widget.show()
app.exec_()

qt.qpa.fonts: Populating font family aliases took 497 ms. Replace uses of missing font family "微软雅黑" with one that exists to avoid this cost. 
